# 3.2 LangChain에서 도구(tool) 활용 방법

- LangChain에서 도구(tool)을 활용하는 방법을 알아봅니다
- 이 강의에서는 도구를 활용하는 방법을 중점적으로 다루며, 도구를 활용한 에이전트 개발 방법은 3.3 강의에서 다룹니다
    - LangGraph에서의 도구 활용 방법은 LangChain의 문법을 따르니 주의깊게 봐주세요

In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o")

small_llm = ChatOpenAI(model="gpt-4o-mini")

- [tool decorator](https://python.langchain.com/docs/how_to/custom_tools/#tool-decorator)를 사용하면 쉽게 도구를 만들 수 있습니다

In [3]:
from langchain_core.tools import tool

@tool
def add(a: int, b: int) -> int:
    """숫자 a와 b를 더합니다."""
    return a + b

@tool
def multiply(a: int, b: int) -> int:
    """숫자 a와 b를 곱합니다."""
    return a * b

- LLM을 호출했을 때와 도구를 사용했을 때의 차이를 알아봅니다

In [4]:
query = '3 곱하기 5는?'

In [5]:
small_llm.invoke(query)

AIMessage(content='3 곱하기 5는 15입니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 15, 'total_tokens': 26, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-C8fPPqcwpPjSWPTICnCNE20dIR3hK', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--162772f8-8625-4ae8-ab2f-68e7d32205d9-0', usage_metadata={'input_tokens': 15, 'output_tokens': 11, 'total_tokens': 26, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

- 도구 리스트는 LLM에 해당하는 `BaseModel` 클래스에 `bind_tools` 메서드를 통해 전달합니다

In [6]:
llm_with_tools = small_llm.bind_tools([add, multiply])

- `AIMessage`의 `additional_kwargs` 속성은 `tool_calls`를 포함합니다
- `tool_calls`는 도구를 호출하는 메시지를 포함합니다

In [7]:
result = llm_with_tools.invoke(query)
result

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_zypvlVC0pfat3nSDifn0CSDJ', 'function': {'arguments': '{"a":3,"b":5}', 'name': 'multiply'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 84, 'total_tokens': 101, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-C8fPQMYzoDrudj558T7sbnJjsBzl3', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--3bb56a8a-c5f6-466a-82c3-409dfe752835-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 5}, 'id': 'call_zypvlVC0pfat3nSDifn0CSDJ', 'type': 'tool_call'}], usage_metadata={'input_tokens': 84, 'output_tokens': 17, 'total_tokens': 101, 'input_token_details': {'audio': 0, 'ca

- 여기서 `tool_calls`의 형태를 기억해두시면 남은 강의를 이해하시는데 도움이 됩니다

In [8]:
result.tool_calls

[{'name': 'multiply',
  'args': {'a': 3, 'b': 5},
  'id': 'call_zypvlVC0pfat3nSDifn0CSDJ',
  'type': 'tool_call'}]

In [9]:
from typing import Sequence

from langchain_core.messages import AnyMessage, HumanMessage

human_message = HumanMessage(query)
message_list: Sequence[AnyMessage] = [human_message] 


- `tool_calls` 속성은 도구를 호출하는 메시지를 포함합니다
- `tool_calls`를 가진 `AIMessage`의 형태를 기억해두시면 남은 강의를 이해하시는데 도움이 됩니다

In [10]:
ai_message = llm_with_tools.invoke(message_list)
ai_message

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_VWAcrqvkDvF4R4aDLD7ywI27', 'function': {'arguments': '{"a":3,"b":5}', 'name': 'multiply'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 84, 'total_tokens': 101, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-C8fPS3VFnpwNfllK9fFaKYHpiy5ZV', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--8bf22297-e2cd-48b2-82f4-2fd72f5934c7-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 5}, 'id': 'call_VWAcrqvkDvF4R4aDLD7ywI27', 'type': 'tool_call'}], usage_metadata={'input_tokens': 84, 'output_tokens': 17, 'total_tokens': 101, 'input_token_details': {'audio': 0, 'ca

In [11]:
ai_message.tool_calls

[{'name': 'multiply',
  'args': {'a': 3, 'b': 5},
  'id': 'call_VWAcrqvkDvF4R4aDLD7ywI27',
  'type': 'tool_call'}]

In [12]:
message_list.append(ai_message)

- `AIMessage`의 `tool_calls`를 활용해서 도구를 직접 호출할 수도 있습니다

In [13]:
tool_message = multiply.invoke(ai_message.tool_calls[0])

In [14]:
tool_message

ToolMessage(content='15', name='multiply', tool_call_id='call_VWAcrqvkDvF4R4aDLD7ywI27')

- 하지만 에이전트의 경우 도구를 직접 호출하는 것이 아니라 도구를 호출하는 메시지를 만들어서 전달합니다

In [15]:
message_list.append(tool_message)

In [16]:
llm_with_tools.invoke(message_list)

AIMessage(content='3 곱하기 5는 15입니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 12, 'prompt_tokens': 109, 'total_tokens': 121, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-C8fPScVWj4EWdOMAmhzLUCwkjeQqN', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--0bfabafc-3596-4f92-a6c9-5475a57679d3-0', usage_metadata={'input_tokens': 109, 'output_tokens': 12, 'total_tokens': 121, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

- `message_list`의 순서를 기억해두시면 남은 강의를 이해하시는데 도움이 됩니다

In [17]:
message_list

[HumanMessage(content='3 곱하기 5는?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_VWAcrqvkDvF4R4aDLD7ywI27', 'function': {'arguments': '{"a":3,"b":5}', 'name': 'multiply'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 84, 'total_tokens': 101, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-C8fPS3VFnpwNfllK9fFaKYHpiy5ZV', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--8bf22297-e2cd-48b2-82f4-2fd72f5934c7-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 5}, 'id': 'call_VWAcrqvkDvF4R4aDLD7ywI27', 'type': 'tool_call'}], usage_metadata={'input_tokens': 84, 